In [83]:
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split


//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


a.       You'll need to  create your test set using stratified sampling, to preserve the distribution of recipes falling into different classes

b.       You may also want to consider the cooking method as a feature. For instance, Chinese cooking rarely (if ever?) requires baking.

c.       You may want to incorporate a cost matrix to assign different penalties for wrongly classifying a recipe as coming from a cuisine that is 'very different' from it's actual cuisine.  

d.       What would the misclassification rate be if you were to randomly assign a recipe to a cuisine, according to the proportion of recipes in the training data that fell into that cuisine? This would be a good baseline against which to compare the performance of your models.

In [12]:
import pandas as pd
train_set = pd.read_json('train.json') 
ingred = []
for x in train_set['ingredients']:
    for ingre in x:
        ingred.append(ingre)

ingredients = set(ingred)

(39774, 3)

In [123]:
len(ingredients)

6714

In [66]:
test_set = pd.read_json('test.json') 
test_set.shape

(9944, 2)

In [4]:
print(type(train_set['ingredients'])) 
train_set.head() 

<class 'pandas.core.series.Series'>


,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [5]:
#Totally 20 types of cusines
len(train_set['cuisine'].unique()) 

20

In [6]:
# distribution of cuisine in dataset
original_dist = (train_set['cuisine'].value_counts() / len(train_set)).to_frame();original_dist

,cuisine
italian,0.197063
mexican,0.161865
southern_us,0.108614
indian,0.075502
chinese,0.067205
french,0.066526
cajun_creole,0.038870
thai,0.038694
japanese,0.035777
greek,0.029542


In [7]:
train_set['cuisine'].unique()  

array(['greek', 'southern_us', 'filipino', 'indian', 'jamaican', 'spanish',
       'italian', 'mexican', 'chinese', 'british', 'thai', 'vietnamese',
       'cajun_creole', 'brazilian', 'french', 'japanese', 'irish',
       'korean', 'moroccan', 'russian'], dtype=object)

In [75]:
ingredients = train_set['ingredients']
ingredients_list = [' '.join(x) for x in ingredients]


In [73]:
train_set['ingredients'][0]

['romaine lettuce',
 'black olives',
 'grape tomatoes',
 'garlic',
 'pepper',
 'purple onion',
 'seasoning',
 'garbanzo beans',
 'feta cheese crumbles']

## Bag of words

In [76]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english',lowercase = True)
X_train_tdm = count_vect.fit_transform(ingredients_list) 
X_train_tdm.shape

(39774, 2970)

In [77]:
X_train, X_valid, y_train, y_valid = train_test_split( 
    X_train_tdm,train_set['cuisine'], test_size=0.25, random_state=0) 

In [78]:
X_train.shape

(29830, 2970)

In [83]:
count_vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [11]:
labelencoder = LabelEncoder()
y_train = labelencoder.fit_transform(y_train)
y_valid = labelencoder.transform(y_valid) 

### Stratified sampling, to preserve the distribution of recipes falling into different classes

In [12]:
train_distribution = (pd.Series(labelencoder.inverse_transform(y_train)).value_counts()/ len(y_train)).to_frame()
test_distribution = (pd.Series(labelencoder.inverse_transform(y_valid)).value_counts() / len(y_valid)).to_frame()
cuisine_dist = (pd.concat([train_distribution, test_distribution,original_dist], axis=1))
cuisine_dist.columns = ['train','test','original'] ;cuisine_dist


,train,test,original
brazilian,0.011633,0.012068,0.011741
british,0.020315,0.019912,0.020214
cajun_creole,0.039692,0.036404,0.038870
chinese,0.066276,0.069992,0.067205
filipino,0.019242,0.018202,0.018982
french,0.066644,0.066171,0.066526
greek,0.029266,0.030370,0.029542
indian,0.074422,0.078741,0.075502
irish,0.017030,0.015990,0.016770
italian,0.197989,0.194288,0.197063


In [13]:
train_distribution=(pd.Series(labelencoder.inverse_transform(y_train)).value_counts()/ len(y_train)).to_frame()
train_distribution.T

,italian,mexican,southern_us,indian,french,chinese,cajun_creole,thai,japanese,greek,spanish,korean,moroccan,vietnamese,british,filipino,irish,jamaican,russian,brazilian
0,0.197989,0.161146,0.10875,0.074422,0.066644,0.066276,0.039692,0.038351,0.036038,0.029266,0.024707,0.02169,0.020851,0.020818,0.020315,0.019242,0.01703,0.012705,0.012437,0.011633


In [14]:
test_distribution = (pd.Series(labelencoder.inverse_transform(y_valid)).value_counts() / len(y_valid)).to_frame()
test_distribution.T

,italian,mexican,southern_us,indian,chinese,french,thai,cajun_creole,japanese,greek,spanish,vietnamese,moroccan,british,korean,filipino,irish,jamaican,brazilian,russian
0,0.194288,0.164019,0.108206,0.078741,0.069992,0.066171,0.039722,0.036404,0.034996,0.03037,0.025342,0.020515,0.020012,0.019912,0.018403,0.018202,0.01599,0.014783,0.012068,0.011866


In [ ]:
from sklearn.linear_model import LogisticRegression
C_penalty = [10**i for i in range(-3,3)]
for C_para in C_penalty:
    LogReg = LogisticRegression(C=C_para)
    LogReg = LogReg.fit(X_train,y_train)
    print('C = ',C_para, 'Score:',LogReg.score(X_valid, y_valid) ) 

C =  0.001 Score: 0.595032180209
C =  0.01 Score: 0.7252614642


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

clf_NB = MultinomialNB(alpha=0.01, fit_prior = True)
clf_NB.fit(X_train, y_train) 

predict_nb = clf_NB.predict(X_valid) 
print (metrics.accuracy_score(y_valid, predict_nb)) 

In [ ]:
from sklearn.svm import SVC
svc_clf = SVC(kernel='linear')
svc_clf.fit(X_train, y_train)
svc_clf.score(X_valid, y_valid)

In [15]:
train_set['cuisine'][:15]

0           greek
1     southern_us
2        filipino
3          indian
4          indian
5        jamaican
6         spanish
7         italian
8         mexican
9         italian
10        italian
11        chinese
12        italian
13        mexican
14        italian
Name: cuisine, dtype: object

## TF-IDF

In [51]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_tdm)
X_train_tfidf.shape 

(39774, 328053)

In [53]:
X_train_tfidf, X_valid_tfidf, y_train_tfidf, y_valid_tfidf = train_test_split( 
    X_train_tfidf,train_set['cuisine'], test_size=0.25, random_state=0) 

In [56]:
labelencoder = LabelEncoder()
y_train_tfidf = labelencoder.fit_transform(y_train)
y_valid_tfidf = labelencoder.transform(y_valid) 

In [57]:
C_penalty = [10**i for i in range(-3,3)]
for C_para in C_penalty:
    LogReg = LogisticRegression(C=C_para)
    LogReg = LogReg.fit(X_train_tfidf,y_train_tfidf)
    print('C = ',C_para, 'Score:',LogReg.score(X_valid_tfidf, y_valid_tfidf) ) 

C =  0.001 Score: 0.194288012872
C =  0.01 Score: 0.321198712792
C =  0.1 Score: 0.567880128721
C =  1 Score: 0.731798069187
C =  10 Score: 0.782280772325
C =  100 Score: 0.787208366854


# Mutual Information

In [84]:
import json
json = json.load(open('train.json'))
#dictB = json.load(open('test.json'))
#merged_dict = {key: value for (key, value) in (dictA.items() + dictB.items())}
# string dump of the merged dict
#jsonString_merged = json.dumps(merged_dict)


In [99]:
dictCuisineIngred = {}
cuisines = []
ingredients = []
    
for i in range(len(json)):
    cuisine = json[i]['cuisine']
    ingredientsPerCuisine = json[i]['ingredients']
    if cuisine not in dictCuisineIngred.keys():
        cuisines.append(cuisine)
        dictCuisineIngred[cuisine] = ingredientsPerCuisine
    else: 
        currentList = dictCuisineIngred[cuisine]
        currentList.extend(ingredientsPerCuisine)
        dictCuisineIngred[cuisine] = currentList
        ingredients.extend(ingredientsPerCuisine)
         
ingredients = list(set(ingredients)) # unique list of ALL ingredients
numUniqueIngredients = len(ingredients)
numCuisines = len(cuisines)

In [17]:
import numpy as np
import pandas as pd
train_set = pd.read_json('train.json') 
ingred = []
for x in train_set['ingredients']:
    for ingre in x:
        ingred.append(ingre)

ingredients = set(ingred)
ingredient_matrix = pd.DataFrame(np.zeros([train_set.shape[0],len(ingredients)]),columns = list(ingredients))
for i in range(train_set.shape[0]):
    for ingredient in train_set.iloc[i][2]:
        ingredient_matrix.iloc[i][ingredient] +=1

In [20]:
import sklearn.feature_selection
#calculate the mutual information of all the variables 
mutual_info = sklearn.feature_selection.mutual_info_classif(ingredient_matrix,train_set['cuisine'])
mu = mutual_info.argsort()
mutual_information = []
#for i in range(len(mu)):
#    mutual_information[i] = mutual_info[mu[i]]
for i in range(len(mu)):
    mutual_information.append([mutual_info[mu[i]],mu[i]])
top_mutual = (mutual_info.shape - sum(mutual_info == 0))
index = [x[1] for x in mutual_information[-top_mutual:]]
#[(list(ingredients)[ind],mutual_info[ind]) for ind in index]
ingred_mutual = [list(ingredients)[ind] for ind in index]
#cleansed_ind_matrix = ingredient_matrix[ingred_mutual]

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


In [27]:
ingred_mutual = [list(ingredients)[ind] for ind in index]

In [29]:
ingred_mutual[::-1]

['soy sauce',
 'sesame oil',
 'fish sauce',
 'ground cumin',
 'olive oil',
 'chili powder',
 'grated parmesan cheese',
 'garam masala',
 'fresh ginger',
 'ginger',
 'corn starch',
 'ground turmeric',
 'rice vinegar',
 'corn tortillas',
 'extra-virgin olive oil',
 'sour cream',
 'all-purpose flour',
 'butter',
 'lime',
 'salsa',
 'avocado',
 'jalapeno chilies',
 'cumin seed',
 'tumeric',
 'scallions',
 'coconut milk',
 'beansprouts',
 'sugar',
 'mirin',
 'unsalted butter',
 'dried oregano',
 'black beans',
 'lemongrass',
 'flour tortillas',
 'ground coriander',
 'chopped cilantro fresh',
 'fresh basil',
 'green onions',
 'green chilies',
 'buttermilk',
 'lime juice',
 'parmesan cheese',
 'cajun seasoning',
 'cumin',
 'hoisin sauce',
 'sake',
 'garlic',
 'onions',
 'sesame seeds',
 'fresh parsley',
 'oil',
 'curry powder',
 'tomatoes',
 'baking powder',
 'fresh lime juice',
 'cilantro',
 'vegetable oil',
 'cilantro leaves',
 'dried thyme',
 'milk',
 'shallots',
 'feta cheese crumbles',
 

In [141]:
for i in range(len(mu)):
    mutual_information.append([mutual_info[mu[i]],mu[i]])

In [30]:
top_mutual = (mutual_info.shape - sum(mutual_info == 0))
index = [x[1] for x in mutual_information[-top_mutual:]]
[(list(ingredients)[ind],mutual_info[ind]) for ind in index]
cleansed_ind_matrix = ingredient_matrix[ingred_mutual]

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  from ipykernel import kernelapp as app


In [33]:
cleansed_ind_matrix.to_csv("cleansed_ind_matrix.csv")

In [165]:
ingred_mutual = [list(ingredients)[ind] for ind in index]

In [167]:
cleansed_ind_matrix = ingredient_matrix[ingred_mutual]

In [90]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(cleansed_ind_matrix)
X_train_tfidf.shape 

(39774, 3787)

In [91]:
X_train_tfidf, X_valid_tfidf, y_train_tfidf, y_valid_tfidf = train_test_split( 
    X_train_tfidf,train_set['cuisine'], test_size=0.25, random_state=0) 

In [94]:
from sklearn.linear_model import LogisticRegression
labelencoder = LabelEncoder()
y_train_tfidf = labelencoder.fit_transform(y_train_tfidf)
y_valid_tfidf = labelencoder.transform(y_valid_tfidf)
C_penalty = [10**i for i in range(-3,3)]
for C_para in C_penalty:
    LogReg = LogisticRegression(C=C_para)
    LogReg = LogReg.fit(X_train_tfidf,y_train_tfidf)
    print('C = ',C_para, 'Score:',LogReg.score(X_valid_tfidf, y_valid_tfidf) ) 

C =  0.001 Score: 0.252111826227
C =  0.01 Score: 0.443785197104
C =  0.1 Score: 0.65577232502
C =  1 Score: 0.760257441673
C =  10 Score: 0.774034593725
C =  100 Score: 0.758346741754


# Random Forest

In [92]:
from sklearn.ensemble import RandomForestClassifier
labelencoder = LabelEncoder()
y_train_tfidf = labelencoder.fit_transform(y_train_tfidf)
y_valid_tfidf = labelencoder.transform(y_valid_tfidf)
#random_forest = RandomForestClassifier()
#random_forest.fit_transform()

In [96]:
random_forest = RandomForestClassifier(n_estimators=500, oob_score=True)
random_forest.fit_transform(X_train_tfidf,y_train_tfidf)
random_forest.score(X_valid_tfidf, y_valid_tfidf)

//anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


0.70595333869670152

In [98]:
# Setup a dataframe -- just like R
df = pd.DataFrame()
df['truth'] = y_valid_tfidf
df['predict'] = random_forest.predict(X_valid_tfidf)

# Cross-tabulate predictions
print(pd.crosstab(df['truth'], df['predict'], margins=True))

predict   0   1    2    3    4    5    6    7   8     9  ...    11   12    13  \
truth                                                    ...                    
0        38   0    0    1    1    2    0    3   0    21  ...     0    0    27   
1         0  32    0    2    0   28    2    7   8    38  ...     0    0     7   
2         0   1  229    1    0   11    0    4   0    32  ...     0    0    30   
3         0   0    0  598    3    6    2    6   1    15  ...    17    8     5   
4         2   0    2   34   75    9    0    5   0    15  ...     0    1    11   
5         1   2    6    6    3  295    1    2   4   218  ...     2    0    15   
6         0   1    0    2    0    7  136    5   1   113  ...     1    0    11   
7         0   0    0    4    1    3    1  687   1    16  ...     0    0    43   
8         1   0    1    2    1   23    0    1  32    17  ...     0    0     5   
9         2   1    3    3    1   62   13    3   0  1736  ...     3    0    36   
10        2   0    4   13   

# FUZZYWUZZY

In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

//anaconda/lib/python3.5/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
import numpy as np
import pandas as pd
train_set = pd.read_json('train.json') 
ingred = []
for x in train_set['ingredients']:
    for ingre in x:
        ingred.append(ingre)

ingredients = set(ingred)

In [8]:
choices = list(ingredients)
process.extract("olive oil", choices)

[('olive oil', 100),
 ('lemon olive oil', 90),
 ('olive oil flavored cooking spray', 90),
 ('virgin olive oil', 90),
 ('olive oil mayonnaise', 90)]

In [21]:
process.extract(ingred_mutual[-1], ingred_mutual)

[('soy sauce', 100),
 ('usukuchi soy sauce', 90),
 ('wheat free soy sauce', 90),
 ('Kikkoman Soy Sauce', 90),
 ('tamari soy sauce', 90)]

In [26]:
mutual_information[-1]

[0.13684931429717873, 5505]

In [49]:
single_ingre = [ingre for ingre in list(ingred_mutual[::-1]) if len(ingre.split(' ')) == 1]

In [66]:
process.extract('salt', ingred_mutual)

[('salt', 90),
 ('salted dry roasted peanuts', 75),
 ('organic unsalted butter', 75),
 ('unsalted almonds', 75),
 ('salted mixed nuts', 75)]